In [1]:
import torch
import torch.nn as nn
from torch.functional import F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
else:
    print ("MPS device not found.")

In [3]:
dataset = load_iris()

In [4]:
X = dataset.data.astype("float32")
y = dataset.target.astype("int64")
y = F.one_hot(torch.from_numpy(y), num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train = torch.from_numpy(X_train).to(mps_device)
y_train = y_train.to(dtype=torch.float32, device=mps_device)
X_test = torch.from_numpy(X_test).to(mps_device)

In [5]:
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [6]:
model = nn.Sequential(nn.Linear(4, 4, dtype=torch.float32),
                      nn.Sigmoid(),
                      nn.Linear(4, 3, dtype=torch.float32),
                      nn.Sigmoid(),
                      nn.Linear(3, 3, dtype=torch.float32),
                      nn.Softmax(dim=-1)).to(mps_device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [11]:
num_epochs = 5000
for epoch in range(num_epochs):
    for data in dataloader:
        optimizer.zero_grad()
        feature, target = data
        pred = model(feature)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()

In [12]:
model(X_test[0])

tensor([8.4121e-05, 9.5781e-02, 9.0414e-01], device='mps:0',
       grad_fn=<SoftmaxBackward0>)

In [10]:
y_test[0]

tensor([0, 0, 1])